This file intends to solve the following questions:
    - build the validation process by comparing brand-profile

In [148]:
import requests 
import os,glob
from lxml import html
from bs4 import BeautifulSoup, __version__
import re #regular expression library
import pandas as pd
import numpy as np
import random
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance
import statistics 

# Import the relative comprehensive brand list
From nordstrom.com (includes women and men)

In [2]:
with open('/Users/hellofutrue/Desktop/Insight/Python/Untitled Folder/brandlist.txt') as f:
    brands = f.read().splitlines()
brands = set(brands) #remove duplicates
brands = list(filter(None, brands)) #remove the empty element

In [3]:
len(brands) #1002 brands in total

1002

In [4]:
voc = dict(zip(brands,range(len(brands))))# create the dictionationary based on brand list
vectorizer = CountVectorizer(vocabulary = voc) #define the vectorizer function

# Import the influencers' posts

In [5]:
# All influencers
posts_influencers = pd.read_csv('/Users/hellofutrue/Desktop/Insight/Python/Feb/files/posts_influencers.csv')
posts_influencers = posts_influencers.rename(index=str, columns={'Unnamed: 0': "people", '0': 'content'})

In [6]:
# import top list
#top influencers list
w = open('/Users/hellofutrue/Desktop/Insight/Python/Feb/files/top_influencer_list.txt', 'r')
list = w.readlines()
w.close()
top_list = list[0].split()

In [7]:
# top influencers only
top_posts = posts_influencers.loc[posts_influencers['people'].isin(top_list)]

In [8]:
top_listnew = top_posts['people'] # 49 top influencers

In [148]:
#write out top_listnew
file = open('/Users/hellofutrue/Desktop/Insight/Python/Feb/top_listnew.txt','w') 
file.write(' '.join(top_listnew)) 
file.close() 

In [153]:
#read in top_listnew
f = open('/Users/hellofutrue/Desktop/Insight/Python/Feb/top_listnew.txt', 'r')
list = f.readlines()
f.close()
top_listnew = list[0].split()

# Vectorizing

In [9]:
# Create brand vector
X = vectorizer.fit_transform(top_posts['content'])

In [10]:
df = pd.DataFrame(X.todense(),index = [top_listnew],columns = [brands]).T

In [11]:
df.head()

people,_emmachamberlain,adele,arianagrande,badgalriri,barackobama,beyonce,blakelively,bretmanrock,champagnepapi,chrisbrownofficial,...,sommerray,stephencurry30,theellenshow,therock,vancityreynolds,wherearetheavocados,willsmith,world_record_egg,zacefron,zendaya
La Vie Rebecca Taylor,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bony Levy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sophie Bille Brahe,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
OUR LEGACY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Urban Decay,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#remove multiindex
df.columns = df.columns.map(''.join)
df.index = df.index.map(''.join)

In [16]:
df.describe()

,_emmachamberlain,adele,arianagrande,badgalriri,barackobama,beyonce,blakelively,bretmanrock,champagnepapi,chrisbrownofficial,...,sommerray,stephencurry30,theellenshow,therock,vancityreynolds,wherearetheavocados,willsmith,world_record_egg,zacefron,zendaya
count,1002.0,1002.0,1002.0,1002.0,1002.0,1002.0,1002.0,1002.0,1002.0,1002.0,...,1002.0,1002.0,1002.000000,1002.000000,1002.0,1002.0,1002.0,1002.0,1002.0,1002.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000998,0.026946,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.031591,0.821947,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.000000,26.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
df.columns[df.sum(axis=0) != 0] #column-wise sum ==> influencers who have mentioned any of the brand

Index(['jamescharles', 'jlo', 'kendalljenner', 'kyliejenner', 'nickiminaj',
       'theellenshow', 'therock'],
      dtype='object')

In [29]:
df.sum(axis=1) #row-wise sum ==>brands who have been talked about

La Vie Rebecca Taylor    0
Bony Levy                0
Sophie Bille Brahe       0
OUR LEGACY               0
Urban Decay              0
Ingrid & Isabel          0
Sorel                    0
Charles David            0
Y-3                      0
Saturday Skin            0
Vera Wang                0
Burberry                 0
Jouer                    0
3LAB                     0
Bogs                     0
Maje                     0
Fitbit                   0
Daniela Villegas         0
Keepsake the label       0
KARA                     0
Shinola                  0
Rebecca Taylor           0
Comme des Garçons        0
beautyblender®           0
Charlotte Chesnais       0
Billy Reid               0
Saloni                   0
7 For All Mankind®       0
Belly Bandit®            0
TOCCA                    0
                        ..
IPKN                     0
Chelsea28                0
Yummie Tummie            0
Dunhill                  0
Munro                    0
Sanctuary                0
L

In [35]:
for column in df.columns[df.sum(axis=0) != 0]:
    a = df[column][df.sum(axis=1) != 0] #row-wise sum
    print(a)

adidas        3
kai           0
philosophy    0
Name: jamescharles, dtype: int64
adidas        1
kai           0
philosophy    0
Name: jlo, dtype: int64
adidas        3
kai           0
philosophy    0
Name: kendalljenner, dtype: int64
adidas        6
kai           0
philosophy    0
Name: kyliejenner, dtype: int64
adidas        0
kai           0
philosophy    1
Name: nickiminaj, dtype: int64
adidas        1
kai           0
philosophy    0
Name: theellenshow, dtype: int64
adidas         0
kai            1
philosophy    26
Name: therock, dtype: int64


Identified the special case: row 42

# Summary
- example of brand equivalence: people who talked about 'adidas'
- jlo
- jamescharles
- kendalljenner 
- kyliejenner

In [40]:
## function
vector = pd.read_csv('/Users/hellofutrue/Desktop/Insight/Python/Feb/files/vector.csv')
vector = vector.set_index('Unnamed: 0') #index = influencer's name

#import outside fanbase info
f = open('/Users/hellofutrue/Desktop/Insight/Python/Feb/files/growing_fashion_influencer_list.txt', 'r')
list = f.readlines()
f.close()
candid_list = list[0].split()

#top influencers list
w = open('/Users/hellofutrue/Desktop/Insight/Python/Feb/files/top_influencer_list.txt', 'r')
list = w.readlines()
w.close()
top_list = list[0].split()

def outputtable1(nameinput):
    dst = pd.DataFrame()
    for people in candid_list:
        row = vector.loc[nameinput, : ].values
        other = vector.loc[people, : ].values
        score = pd.DataFrame(distance.euclidean(row, other),index=[people], columns=['Euclidean Distance'])
        dst = dst.append(score)
    result = dst.sort_values(['Euclidean Distance'], ascending=[1])[1:6]
    #for name in result.index:
    #    result.set_value(name, 'fanbase',database.loc[database['name'] == name]['fanbase'])
    result = result.assign(Similarity_Index = result.iloc[:,0])
    result.loc[result['Similarity_Index'] <= 0.001, 'Similarity_Index'] = 'Extremely similar' 
    return result

In [92]:
# no overlapp results btw any of the four candidates
outputtable1('kyliejenner').index.intersection(outputtable1('kendalljenner').index)

Index([], dtype='object')

In [113]:
def validationsys(nameinput1, nameinput2):
    row = vector.loc[nameinput1, : ].values
    other = vector.loc[nameinput2, : ].values
    score = distance.euclidean(row, other)
    return score

In [118]:
validationsys('jlo','kendalljenner')

0.037212236088145924

In [115]:
validationsys('jlo','jamescharles')

0.0421409943755226

In [107]:
validationsys('jlo','kyliejenner')

,kyliejenner
jlo,0.078149


In [108]:
validationsys('kendalljenner','jamescharles')

,jamescharles
kendalljenner,0.015534


In [109]:
validationsys('kyliejenner','jamescharles')

,jamescharles
kyliejenner,0.070077


In [110]:
validationsys('kendalljenner','kyliejenner')

,kyliejenner
kendalljenner,0.058518


In [121]:
l =[validationsys('jlo','kendalljenner'), 
    validationsys('jlo','jamescharles'),
    validationsys('jlo','kyliejenner'),
    validationsys('kendalljenner','jamescharles'),
    validationsys('kyliejenner','jamescharles'),
    validationsys('kendalljenner','kyliejenner')]
print(np.mean(l))

0.05027198449984949


# similarity score of any of them with people outside

In [123]:
namelist = ['jlo','jamescharles','kyliejenner','kendalljenner']

In [150]:
fulllist = df.columns.tolist()

In [142]:
fulllist.remove('jlo')
fulllist.remove('jamescharles')
fulllist.remove('kyliejenner')
fulllist.remove('kendalljenner')

In [144]:
outsidelist = fulllist

In [153]:
slist = []
for people in ['jlo']:
    for another in outsidelist:
        simi = validationsys(people, another)
        slist.append(simi)
print(statistics.mean(slist))

0.05400510959952268


In [154]:
(0.05400510959952268- 0.05027198449984949)/0.05027198449984949

0.07425855845584069